In [1]:
import os
from keras.layers import Input, Dense, Flatten, Reshape
from sklearn.feature_extraction import image as sk_image
from concurrent.futures import ProcessPoolExecutor
from keras.models import Model
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from sklearn.cluster import AgglomerativeClustering
from scipy.signal import convolve2d
from sklearn.cluster import KMeans
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from pyhdf.SD import SD, SDC
import matplotlib as mpl
#tf.config.threading.set_inter_op_parallelism_threads(1)
from extract_training_data import *
from sklearn.feature_extraction.image import extract_patches_2d, reconstruct_from_patches_2d
from pyhdf.error import HDF4Error
from functions import *
from tensorflow.keras.mixed_precision import experimental as mixed_precision
from sklearn.model_selection import train_test_split

import random


total cores: 256


In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

# policy = mixed_precision.Policy('mixed_float16')
# mixed_precision.set_policy(policy)

#bands = [6, 7, 20, 28, 28, 31]


1 Physical GPUs, 1 Logical GPUs


2023-09-28 12:15:43.858833: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 21336 MB memory:  -> device: 0, name: Quadro RTX 6000, pci bus id: 0000:25:00.0, compute capability: 7.5


In [3]:
#bands = [6,20,29]
bands = [6,7,20,28,29,31]


In [4]:
### PROCESSING .hdf files
import extract_training_data
import importlib
importlib.reload(extract_training_data)
from extract_training_data import *
#folder = "/uio/hume/student-u37/fslippe/data/training_data/MOD02QKM/normalized_data/"
folder = "/scratch/fslippe/modis/MOD02/daytime_1km/"

#folder = "/uio/hume/student-u37/fslippe/data/nird_mount/MOD02QKM_202012-202104/normalized_data/"

start = "20201201"
end = "20210416"
#end = "20201230"

#dates = ["20210303"]#, "20210322"]#, "20210323"]
#dates_converted = []
#for date in dates:
#    dates_converted.append(convert_to_day_of_year(date))
#bands = [6,20,29]

start_converted = convert_to_day_of_year(start)
end_converted = convert_to_day_of_year(end)
#print(start_converted)
#print(end_converted)
#x = [xi for xi in  extract_250m_data(folder, bands=[1], date_list=dates_converted) if xi.shape[0] > 256]
x = [xi for xi in  extract_1km_data(folder, bands=bands, start_date=start_converted, end_date=end_converted) if xi.shape[0] > 64]

len(x)
#x = extract_250m_data(folder, bands=[1], start_date=start_converted, end_date=end_converted)


total cores: 256
/scratch/fslippe/modis/MOD02/daytime_1km/MOD021KM.A2020336.0800.061.2020337214455.hdf
0 Latitude
1 Longitude
2 EV_1KM_RefSB
3 EV_1KM_RefSB_Uncert_Indexes
4 EV_1KM_Emissive
5 EV_1KM_Emissive_Uncert_Indexes
6 EV_250_Aggr1km_RefSB
7 EV_250_Aggr1km_RefSB_Uncert_Indexes
8 EV_250_Aggr1km_RefSB_Samples_Used
9 EV_500_Aggr1km_RefSB
10 EV_500_Aggr1km_RefSB_Uncert_Indexes
11 EV_500_Aggr1km_RefSB_Samples_Used
12 Height
13 SensorZenith
14 SensorAzimuth
15 Range
16 SolarZenith
17 SolarAzimuth
18 gflags
19 EV_Band26
20 EV_Band26_Uncert_Indexes
21 Band_250M
22 Band_500M
23 Band_1KM_RefSB
24 Band_1KM_Emissive
25 Noise in Thermal Detectors
26 Change in relative responses of thermal detectors
27 DC Restore Change for Thermal Bands
28 DC Restore Change for Reflective 250m Bands
29 DC Restore Change for Reflective 500m Bands
30 DC Restore Change for Reflective 1km Bands


  0%|          | 0/110 [00:00<?, ?it/s]/opt/software/easybuild/software/SciPy-bundle/2021.10-foss-2021b/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/software/easybuild/software/SciPy-bundle/2021.10-foss-2021b/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)
100%|██████████| 110/110 [01:22<00:00,  1.34it/s]


518

In [5]:
import autoencoder
import importlib
importlib.reload(autoencoder)
from autoencoder import SobelFilterLayer, SimpleAutoencoder
patch_size = 64

autoencoder = SimpleAutoencoder(len(bands), patch_size, patch_size)
#x = autoencoder.normalize(x)
#optimizer = mixed_precision.LossScaleOptimizer(tf.keras.optimizers.Adam(learning_rate=1e-4), loss_scale='dynamic')
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)

model = autoencoder.model(optimizer=optimizer, loss="combined")


Input should already be normalized. Call self.normalize to normalize list of data


In [6]:
patches = np.concatenate([autoencoder.extract_patches(n_d) for n_d in x], axis=0)


In [7]:
# Splitting the data
patches, val_data = train_test_split(patches, test_size=0.1, random_state=42)
print("finished split")

finished split


In [8]:
print(patches.shape)
print(val_data.shape)


(300169, 64, 64, 6)
(33353, 64, 64, 6)


In [9]:
min_val = 0
max_val = np.max(patches, axis=(0,1,2), keepdims=True)
print(max_val)
patches = (patches - min_val) / (max_val - min_val)
val_data = (val_data - min_val) / (max_val - min_val)


[[[[54.087894  17.07861    1.8587686  4.011136  15.703261  12.105271 ]]]]


In [11]:
#np.save("/scratch/fslippe/modis/MOD02/test_data/normalized_testpatches_band(1)_winter20_21.npy", val_data)
#np.save("/scratch/fslippe/modis/MOD02/training_data/normalized_trainingpatches_band(1)_winter20_21.npy", patches)
# val_data = np.load("/uio/hume/student-u37/fslippe/data/models/winter_2020_21_val_patches.npy")
# patches = np.load("/uio/hume/student-u37/fslippe/data/models/winter_2020_21_train_patches.npy")[:int(170e3)]


In [10]:
### SAVE TRAIN TEST PATCHES 
# patches = np.load("/scratch/fslippe/modis/MOD02/training_data/normalized_trainingpatches_bands6,20,29_winter20_21.npy")[::4]
# val_data  = np.load("/scratch/fslippe/modis/MOD02/test_data/normalized_testpatches_bands6,20,29_winter20_21.npy")[::4]
# patches.shape

# print(np.mean(patches, axis=(0,1,2)))


float32


In [13]:
### STANDARD FIT
gc.collect()


model.fit(patches[:100000], patches[:100000], epochs=200, batch_size=32)


2023-09-28 12:27:04.130476: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 9830400000 exceeds 10% of free system memory.
2023-09-28 12:27:20.659459: W tensorflow/core/common_runtime/bfc_allocator.cc:462] Allocator (GPU_0_bfc) ran out of memory trying to allocate 9.16GiB (rounded to 9830400000)requested by op _EagerConst
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2023-09-28 12:27:20.659525: I tensorflow/core/common_runtime/bfc_allocator.cc:1010] BFCAllocator dump for GPU_0_bfc
2023-09-28 12:27:20.659539: I tensorflow/core/common_runtime/bfc_allocator.cc:1017] Bin (256): 	Total Chunks: 47, Chunks in use: 47. 11.8KiB allocated for chunks. 11.8KiB in use in bin. 3.6KiB client-requested in use in bin.
2023-09-28 12:27:20.659547: I tensorflow/core/common_runtime/bfc_allocator.cc:1017] Bin (512): 	Total Chunks: 

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

In [ ]:

def data_generator():
    """Generator to yield data from files."""
    file_list = ['file1.npy', 'file2.npy', ...]
    random.shuffle(file_list)  # Shuffle files at the beginning of each epoch
    for file_name in file_list:
        data = np.load(file_name)
        for item in data:
            yield item

# Define your dataset
dataset = tf.data.Dataset.from_generator(data_generator,
                                         output_signature=(tf.TensorSpec(shape=(...), dtype=tf.float32)))  # Fill in the shape and type
dataset = dataset.shuffle(buffer_size=10000)  # Shuffle data
dataset = dataset.batch(32)  # Batch data
dataset = dataset.repeat()  # Repeat dataset indefinitely
dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)  # Prefetch data
steps_per_epoch = ...  

model.fit(dataset, epochs=200, steps_per_epoch=steps_per_epoch, validation_data=(val_data, val_data))

In [ ]:
#model.save("/uio/hume/student-u37/fslippe/data/models/winter_2020_21_autoencoder")
autoencoder.encoder.save("/uio/hume/student-u37/fslippe/data/models/winter_2020_21_band(6,20,29)_encoder")
autoencoder.decoder.save("/uio/hume/student-u37/fslippe/data/models/winter_2020_21_band(6,20,29)_decoder")
